In [1]:
data = open('../inputs/day-15').read().splitlines()

In [2]:
import re

In [3]:
loc = {}

In [4]:
def dist(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

In [5]:
for l in data:
    sx, sy, bx, by = list(map(int, re.match(".* x=(\S+), y=(\S+):.* x=(\S+), y=(\S+)", l).groups([1,2,3,4])))
    loc[(sx,sy)] = (bx,by)

In [6]:
distances = {}
for s, b in loc.items():
    distances[s] = dist(s,b)

In [8]:
y_q = 2_000_000

In [9]:
ans = set()

In [10]:
def solve_sensor(s, L):
    val = L - abs(s[1] - y_q)
    if val > 0:
        ans.update(range(s[0] - val, s[0] + val + 1))

In [11]:
for s, L in distances.items():
    solve_sensor(s, L)

Need to remove the sensors and beacons locations:

In [12]:
ans = ans.difference([s[0] for s in loc.keys() if s[1] == y_q])
ans = ans.difference([s[0] for s in loc.values() if s[1] == y_q])

In [13]:
len(ans)

5688618

## Find the location

The idea is that the only empty location is at the immediate perimeter of the clusters, the L+1 shell, if you like. 

In [15]:
limit = 4_000_000

In [16]:
def get_perim(s, L):
    ans = set()
    sx, sy = s
    py = sy + L + 1
    px = sx
    for _ in range(L + 1):
        if 0 <= px <= limit and 0 <= py <= limit:
            ans.add((px,py))
        px += 1
        py -= 1
    for _ in range(L + 1):
        if 0 <= px <= limit and 0 <= py <= limit:
            ans.add((px,py))
        px -= 1
        py -= 1
    for _ in range(L + 1):
        if 0 <= px <= limit and 0 <= py <= limit:
            ans.add((px,py))
        px -= 1
        py += 1
    for _ in range(L + 1):
        if 0 <= px <= limit and 0 <= py <= limit:
            ans.add((px,py))
        px += 1
        py += 1

    return ans

This empty location lies in the intersection of at least some of the other perimeters (or it's in the corner, which isn't interesting).

In [17]:
ans = set()
shared = set()

This takes about 1 minute:

In [19]:
for s,L in distances.items():
    per = get_perim(s, L)
    shared.update(per.intersection(ans))
    ans.update(per)

In [21]:
len(shared)

1154346

There are 1M such points, so we need to check manually who is the only point which is not any cluster:

In [22]:
def test_p(p):
    for s, b in loc.items():
        L = dist(s,b)
        if dist(p, s) <= L:
            return False
    return True

In [24]:
ans = None

In [25]:
for p in shared:
    if test_p(p):
        ans = p
        break

In [26]:
4000000 * ans[0] + ans[1]

12625383204261